In [22]:
!pip install psycopg2

import psycopg2

username = "sueszli"
password = "password"
host = "localhost"
port = "5432"
database = "postgres"

# test connection
conn = psycopg2.connect(
    user=username,
    password=password,
    host=host,
    port=port,
    database=database
)


In [23]:
!psql postgres -c "\dt"

!psql postgres -c "EXPLAIN SELECT * FROM pg_stat_activity"

           List of relations
 Schema |    Name     | Type  |  Owner  
--------+-------------+-------+---------
 public | badges      | table | sueszli
 public | comments    | table | sueszli
 public | posthistory | table | sueszli
 public | postlinks   | table | sueszli
 public | posts       | table | sueszli
 public | r           | table | sueszli
 public | s           | table | sueszli
 public | t           | table | sueszli
 public | tags        | table | sueszli
 public | users       | table | sueszli
 public | votes       | table | sueszli
(11 rows)

                                        QUERY PLAN                                         
-------------------------------------------------------------------------------------------
 Hash Left Join  (cost=2.38..3.93 rows=100 width=452)
   Hash Cond: (s.usesysid = u.oid)
   ->  Hash Left Join  (cost=1.05..2.32 rows=100 width=388)
         Hash Cond: (s.datid = d.oid)
         ->  Function Scan on pg_stat_get_activity s  (cost=0.00..1